In [1]:
import cv2
import mediapipe as mp
import time
import numpy as np

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils


In [2]:
def finger_is_up(landmarks, tip_idx, pip_idx):
    return landmarks.landmark[tip_idx].y < landmarks.landmark[pip_idx].y

def classify_gesture(landmarks):
    up_index = finger_is_up(landmarks, 8, 6)
    up_middle = finger_is_up(landmarks, 12, 10)
    up_ring = finger_is_up(landmarks, 16, 14)
    up_pinky = finger_is_up(landmarks, 20, 18)
    thumb_up = landmarks.landmark[4].y < landmarks.landmark[3].y

    if up_index and up_middle and up_ring and up_pinky and thumb_up:
        return "Open Palm"
    if (not up_index) and (not up_middle) and (not up_ring) and (not up_pinky):
        if thumb_up:
            return "Thumbs Up"
        return "Fist"
    if thumb_up and (not up_index and not up_middle and not up_ring and not up_pinky):
        return "Thumbs Up"
    return "Unknown"

def gesture_to_action(gesture):
    mapping = {"Open Palm": "PAUSE", "Fist": "PLAY", "Thumbs Up": "LIKE"}
    return mapping.get(gesture, "None")


In [ ]:
# Debugging & Auto-calibration gesture cell (paste into Jupyter)
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='google.protobuf')

import cv2, mediapipe as mp, time, math, pyautogui, numpy as np
from collections import deque, Counter

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# ---------------- PARAMETERS (tweak these) ----------------
MODEL_COMPLEXITY = 1          # 0 = faster, 1 = more accurate
MIN_DET_CONF = 0.6
MIN_TRACK_CONF = 0.6

HISTORY_LEN = 6              # smoothing buffer length
ACTION_COOLDOWN = 1.2        # secs
FIST_REL_THRESH = 0.60       # lower -> stricter fist detection, raise if misses
OK_REL_THRESH = 0.22         # threshold for thumb-index for OK
CALIB_FRAMES = 25            # #frames used for calibration
# ---------------------------------------------------------

# helpers
def euclid_xy(a, b):
    return math.hypot(a.x - b.x, a.y - b.y)

def wrist_to_index_distance(landmarks):
    return euclid_xy(landmarks.landmark[0], landmarks.landmark[8]) + 1e-9

def finger_is_up(landmarks, tip_idx, pip_idx):
    return landmarks.landmark[tip_idx].y < landmarks.landmark[pip_idx].y

# raw classifier (same logic, using REL sizes)
def classify_gesture_raw(landmarks, rel_size):
    up_index = finger_is_up(landmarks, 8, 6)
    up_middle = finger_is_up(landmarks, 12, 10)
    up_ring = finger_is_up(landmarks, 16, 14)
    up_pinky = finger_is_up(landmarks, 20, 18)
    thumb_up = landmarks.landmark[4].y < landmarks.landmark[3].y
    fingers_up = sum([up_index, up_middle, up_ring, up_pinky])

    # distances normalized by rel_size
    d_index = euclid_xy(landmarks.landmark[8], landmarks.landmark[0]) / rel_size
    d_middle = euclid_xy(landmarks.landmark[12], landmarks.landmark[0]) / rel_size
    d_ring = euclid_xy(landmarks.landmark[16], landmarks.landmark[0]) / rel_size
    d_pinky = euclid_xy(landmarks.landmark[20], landmarks.landmark[0]) / rel_size
    d_thumb_index = euclid_xy(landmarks.landmark[4], landmarks.landmark[8]) / rel_size

    # 1) Fist
    if (d_index < FIST_REL_THRESH) and (d_middle < FIST_REL_THRESH) and (d_ring < FIST_REL_THRESH) and (d_pinky < FIST_REL_THRESH):
        return "Fist", {"d_index":d_index,"d_middle":d_middle,"d_ring":d_ring,"d_pinky":d_pinky}
    # 2) Peace
    if up_index and up_middle and (not up_ring) and (not up_pinky):
        return "Peace (V)", {}
    # 3) Pointing
    if up_index and (not up_middle) and (not up_ring) and (not up_pinky):
        return "Pointing", {}
    # 4) OK
    if d_thumb_index < OK_REL_THRESH:
        return "OK", {"d_thumb_index":d_thumb_index}
    # 5) Open Palm
    if fingers_up == 4 and thumb_up:
        return "Open Palm", {}
    # 6) Thumbs up/down
    if (not up_index) and (not up_middle) and (not up_ring) and (not up_pinky):
        thumb_tip = landmarks.landmark[4]
        wrist = landmarks.landmark[0]
        if thumb_tip.y < wrist.y:
            return "Thumbs Up", {}
        else:
            return "Thumbs Down", {}
    return "Unknown", {}

def gesture_to_action(gesture):
    mapping = {
        "Open Palm": "PAUSE",
        "Fist": "None",           # disable Fist action
        "Thumbs Up": "VOL_UP",
        "Thumbs Down": "VOL_DOWN",
        "Peace (V)": "PLAY",      # ✌️ now plays video/music
        "OK": "CONFIRM",
        "Pointing": "SELECT",
        "Unknown": "None"
    }
    return mapping.get(gesture, "None")


def perform_action(action):
    if action == "PLAY" or action == "PAUSE":
        pyautogui.press("space")
    elif action == "NEXT":
        pyautogui.hotkey("ctrl", "right")
    elif action == "VOL_UP":
        pyautogui.press("volumeup")
    elif action == "VOL_DOWN":
        pyautogui.press("volumedown")
    elif action == "CONFIRM":
        pyautogui.press("enter")
    elif action == "SELECT":
        pyautogui.click()

# ---------- state ----------
show_landmarks = True
debug_mode = True   # toggle on-screen numeric debug info
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

gesture_buffers = [deque(maxlen=HISTORY_LEN), deque(maxlen=HISTORY_LEN)]
last_action_time = 0

# calibration storage
calibrating = False
calib_counts = 0
calib_size_accum = 0.0
calibrated_size = None

print("Controls: 'c' to calibrate (capture hand size), 'd' toggle debug, 's' toggle landmarks, 'q' quit.")

with mp_hands.Hands(max_num_hands=2, model_complexity=MODEL_COMPLEXITY,
                    min_detection_confidence=MIN_DET_CONF,
                    min_tracking_confidence=MIN_TRACK_CONF) as hands:
    prev_time = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Camera error.")
            break
        frame = cv2.flip(frame, 1)
        h, w, _ = frame.shape
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb)

        if results.multi_hand_landmarks and results.multi_handedness:
            for idx, (hand_landmarks, hand_handedness) in enumerate(zip(results.multi_hand_landmarks, results.multi_handedness)):
                rel_size = wrist_to_index_distance(hand_landmarks)
                # if user calibrated, scale by that
                rel_scale = calibrated_size if calibrated_size else rel_size

                gesture, debug_vals = classify_gesture_raw(hand_landmarks, rel_scale)
                if idx < 2:
                    gesture_buffers[idx].append(gesture)
                # majority vote
                most_common, count = Counter(gesture_buffers[idx]).most_common(1)[0] if gesture_buffers[idx] else ("Unknown",0)
                smoothed = most_common if count >= (HISTORY_LEN//2) else "Unknown"
                action = gesture_to_action(smoothed)

                # draw
                xs = [lm.x for lm in hand_landmarks.landmark]
                ys = [lm.y for lm in hand_landmarks.landmark]
                x_min, x_max = int(min(xs)*w)-10, int(max(xs)*w)+10
                y_min, y_max = int(min(ys)*h)-30, int(max(ys)*h)+10
                y_min = max(0, y_min)
                if show_landmarks:
                    mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                label_text = f"{hand_handedness.classification[0].label} | {smoothed} | {action}"
                cv2.rectangle(frame, (x_min,y_min), (x_max,y_max), (10,150,10), 2)
                cv2.putText(frame, label_text, (x_min+5, y_min+20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)
                cv2.putText(frame, f"{int(hand_handedness.classification[0].score*100)}%", (x_max-50, y_min+20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200,200,200), 1)

                # debug overlays (numeric)
                if debug_mode:
                    # show rel_size & debug_vals
                    cv2.putText(frame, f"rel_size:{rel_size:.3f}", (x_min+5, y_max+20), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (220,220,0), 1)
                    yline = y_max+40
                    for k,v in debug_vals.items():
                        cv2.putText(frame, f"{k}:{v:.3f}", (x_min+5, yline), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (200,200,200), 1)
                        yline += 18

                # perform action with cooldown
                current_time = time.time()
                if action != "None" and (current_time - last_action_time > ACTION_COOLDOWN):
                    perform_action(action)
                    last_action_time = current_time

        # show calibration status if running
        if calibrating:
            cv2.putText(frame, f"Calibrating... {calib_counts}/{CALIB_FRAMES}", (10,50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,200,200),2)
        elif calibrated_size:
            cv2.putText(frame, f"Calibrated size: {calibrated_size:.3f}", (10,50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (180,180,180),2)

        # fps & hints
        curr_time = time.time()
        fps = 1/(curr_time-prev_time) if prev_time else 0.0
        prev_time = curr_time
        cv2.putText(frame, f"FPS:{int(fps)}", (10,20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,255),2)
        cv2.putText(frame, "q:quit  c:calibrate  d:debug  s:landmarks", (10, h-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200,200,200),1)

        cv2.imshow("Gesture Debugger", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        if key == ord('s'):
            show_landmarks = not show_landmarks
        if key == ord('d'):
            debug_mode = not debug_mode
        if key == ord('c'):
            # start calibration
            calibrating = True
            calib_counts = 0
            calib_size_accum = 0.0
            calibrated_size = None

        # handle calibration accumulation outside loop to avoid missing frames
        if calibrating and results and results.multi_hand_landmarks:
            # accumulate relative size from first detected hand
            first = results.multi_hand_landmarks[0]
            cs = wrist_to_index_distance(first)
            calib_size_accum += cs
            calib_counts += 1
            if calib_counts >= CALIB_FRAMES:
                calibrated_size = calib_size_accum / calib_counts
                calibrating = False
                print(f"Calibration done. calibrated_size = {calibrated_size:.4f}")

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


Controls: 'c' to calibrate (capture hand size), 'd' toggle debug, 's' toggle landmarks, 'q' quit.
